In [ ]:
import os
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets

def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def evaluate_all(data_dir):
    display(H1("Wärmekapazität"))
    
    display(H2("Versuch"))
    # special heat capacity [J / (g K)]
    c_cu = 0.38
    c_h2o = 4.2
    # heat capacity thermometer [J / K]
    Gamma_T = 5
    # mass weight [kg]
    m = 5
    # gravity [m/s^2]
    g = 9.81
    # rotations
    n = 200

    # copper mass [g], total mass [g]
    # temp before heating [°C], temp after heating [°C]
    # diameter of the calorimeter [mm]
    m_cu, m_ges, T_1, T_2, d = np.load(os.path.join(data_dir, "DataV14Part1.npy"))
    
    # Round in the same way as given in the students' tables
    T_1 = round(T_1, 1)
    T_2 = round(T_2, 1)
    m_cu = round(m_cu, 1)
    m_ges = round(m_ges, 1)
    
    # radius [m]
    r = d / 1000 / 2
    # heat capacity calorimeter [J / K]
    Gamma_K = c_cu * (m_cu)
    # mass water [g]
    m_w = m_ges - m_cu
    # temperature difference [K]
    Delta_T = T_2 - T_1

    print("heat capacity calorimeter = {:.2f} J / K".format(Gamma_K))
    print("mass water = {:.2f} g".format(m_w))
    print("temperature difference = {:.2f} K".format(Delta_T))
    # special heat capacity water [J / (g K)]
    c_w = (m * g * n * 2 * np.pi * r - (Gamma_K + Gamma_T) * Delta_T) / (m_w * Delta_T)

    print("special heat capacity water = {:.2f} J / g / K".format(c_w))

    display(H2("Rechenaufgaben"))
    C_human = c_h2o* 70e3
    C_human_exp = c_w * 70e3
    print(f"Wärmekapazität Mensch (cw Literaturwert) {C_human:.0f} J/K")
    print(f"Wärmekapazität Mensch (cw gemessen) {C_human_exp:.0f} J/K")
    
    delta_T = 1  # K
    P = 100  # W
    t = C_human * delta_T / P
    t_exp = C_human_exp * delta_T / P
    print(f"1°C Erwärmung (cw Literaturwert) {t:.0f}s = {t / 60:.0f}min")
    print(f"1°C Erwärmung (cw gemessen) {t_exp:.0f}s = {t_exp / 60:.0f}min")
    
# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V14", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})